In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model,Sequential,load_model
from keras.applications.vgg16 import VGG16 ,preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
Image_size=[224,224]

train_path='/content/drive/My Drive/training'
test_path='/content/drive/My Drive/validation'

In [ ]:
vgg = VGG16(input_shape=Image_size+[3],weights='imagenet',include_top=False)

for layer in vgg.layers:
    layer.trainable=False



x=Flatten()(vgg.output)

first=Dense(2048,activation='relu')(x)
second=Dense(256,activation='relu')(first)
prediction = Dense(11,activation='softmax')(second)

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
model=Model(inputs=vgg.input,outputs=prediction)

model.summary()

model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=.2,
                                zoom_range=.1,
                                 horizontal_flip=True
                                )
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_set=train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set=test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1301 images belonging to 11 classes.
Found 237 images belonging to 11 classes.


In [ ]:
model1=model.fit(train_set,validation_data=test_set,shuffle=True,epochs=10,steps_per_epoch=len(train_set),
  validation_steps=len(test_set))

Epoch 1/10
41/41 [==============================] - 560s 14s/step - loss: 2.4226 - accuracy: 0.3036 - val_loss: 1.7680 - val_accuracy: 0.4135
Epoch 2/10
41/41 [==============================] - 40s 973ms/step - loss: 1.0634 - accuracy: 0.6472 - val_loss: 0.7686 - val_accuracy: 0.7722
Epoch 3/10
41/41 [==============================] - 40s 966ms/step - loss: 0.5899 - accuracy: 0.8263 - val_loss: 0.5675 - val_accuracy: 0.8819
Epoch 4/10
41/41 [==============================] - 40s 970ms/step - loss: 0.4411 - accuracy: 0.8663 - val_loss: 0.5629 - val_accuracy: 0.8481
Epoch 5/10
41/41 [==============================] - 40s 971ms/step - loss: 0.3333 - accuracy: 0.9116 - val_loss: 0.4115 - val_accuracy: 0.8903
Epoch 6/10
41/41 [==============================] - 40s 970ms/step - loss: 0.2028 - accuracy: 0.9539 - val_loss: 0.2965 - val_accuracy: 0.9325
Epoch 7/10
41/41 [==============================] - 40s 966ms/step - loss: 0.1510 - accuracy: 0.9662 - val_loss: 0.2906 - val_accuracy: 0.9283


In [ ]:
 from keras.preprocessing.image import load_img


def preprocess_img(img):
    
    img=image.load_img(img,target_size=(224,224))
    
    img=image.img_to_array(img)
    
    img=np.expand_dims(img,axis=0)
    
    # normalisation
    
    img=preprocess_input(img)
    
    return img

img=preprocess_img("/content/drive/My Drive/validation/50/11.jpg")

label=model.predict(img)
print(label)
print(label.argmax())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
9


In [ ]:
test_set.class_indices

{'1': 0,
 '10': 1,
 '100': 2,
 '1000': 3,
 '2': 4,
 '20': 5,
 '200': 6,
 '2000': 7,
 '5': 8,
 '50': 9,
 '500': 10}

In [ ]:
from keras.models import load_model 
model.save("currency_detection.h5") 